# Recommender Systems 2020/21

### Practice - Hybrid model with LightFM on MovieLens

### Factorization Machines can be used to build different types of models
* Pure Collaborative Filtering
* Content-based
* Hybrid Content-Collaborative
* Heterogeneous Feature models


![FactorizationMachine.png](images/FactorizationMachine.png)

In [1]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()
ICM_genres = data_loaded.get_ICM_from_name("ICM_genres")

Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: <class 'Data_manager.Dataset.Dataset'>
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10126, feature occurrences: 128384, density 1.19E-03
	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10106, feature occurrences: 106820, density 9.90E-04
	ICM name: ICM_year, Value range: 6.00E+00 / 2.01E+03, Num features: 1, feature occurrences: 10681, density 1.00E+00




In [2]:
from Evaluation.Evaluator import EvaluatorHoldout

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 69643 ( 0.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 69803 ( 0.1%) Users that have less than 1 test interactions


## A pure collaborative filtering model

In order to train a pure CF model it is sufficient to provide to the FM the user-item interaction data

In [3]:
## In order to evaluate put it in a recommender class
from Recommenders.BaseRecommender import BaseRecommender
from lightfm import LightFM
import numpy as np

class LightFMCFRecommender(BaseRecommender):
    """LightFMCFRecommender"""

    RECOMMENDER_NAME = "LightFMCFRecommender"

    def __init__(self, URM_train):
        super(LightFMCFRecommender, self).__init__(URM_train)


    def fit(self, epochs = 300, alpha = 1e-6, n_factors = 10, n_threads = 4):
        
        # Let's fit a WARP model
        self.lightFM_model = LightFM(loss='warp',
                                     item_alpha=alpha,
                                     no_components=n_factors)

        self.lightFM_model = self.lightFM_model.fit(URM_train, 
                                       epochs=epochs,
                                       num_threads=n_threads)


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        # Create a single (n_items, ) array with the item score, then copy it for every user
        items_to_compute = np.arange(self.n_items)
        
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf

        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index] = self.lightFM_model.predict(int(user_id), 
                                                                 items_to_compute)

        return item_scores


C:\Users\ferra\Anaconda3\envs\RecSysFramework38\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [4]:
# Set the number of threads; you can increase this
# if you have more physical cores available.

recommender = LightFMCFRecommender(URM_train)
recommender.fit(epochs = 10)

result_df, _ = evaluator_validation.evaluateRecommender(recommender)
result_df

LightFMCFRecommender: URM Detected 63 ( 0.6%) items with no interactions.
EvaluatorHoldout: Processed 69643 (100.0%) in 3.32 min. Users per second: 349


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.183077,0.226539,0.144772,0.095729,0.115251,0.390238,0.17913,0.161686,0.737562,0.601279,...,0.078363,0.996637,0.735081,0.012019,7.35778,0.990365,0.061727,0.649685,1.985386,0.087982


## A content-based model

It is possible to use only item features to try to predict the user-item interactions. LightFM automatically discards ItemIDs when item features are provided. The same happens for UserIDs when the UCM is provided.

#### Warning: this may not be the behaviour you expect

In [7]:
class LightFMCBFRecommender(BaseRecommender):
    """LightFMCBFRecommender"""

    RECOMMENDER_NAME = "LightFMCBFRecommender"

    def __init__(self, URM_train, ICM_train):
        super(LightFMCBFRecommender, self).__init__(URM_train)
        
        self.ICM_train = ICM_train.copy()


    def fit(self, epochs = 300, alpha = 1e-6, n_factors = 10, n_threads = 4):
        
        # Let's fit a WARP model
        self.lightFM_model = LightFM(loss='warp',
                                     item_alpha=alpha,
                                     no_components=n_factors)

        self.lightFM_model = self.lightFM_model.fit(URM_train, 
                                       item_features=self.ICM_train, 
                                       epochs=epochs, 
                                       num_threads=n_threads)


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        # Create a single (n_items, ) array with the item score, then copy it for every user
        items_to_compute = np.arange(self.n_items)
        
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf

        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index] = self.lightFM_model.predict(int(user_id), 
                                                                 items_to_compute,
                                                                 item_features = self.ICM_train)

        return item_scores


In [8]:
recommender = LightFMCBFRecommender(URM_train, ICM_genres)
recommender.fit(epochs = 10)

result_df, _ = evaluator_validation.evaluateRecommender(recommender)
result_df

LightFMCBFRecommender: URM Detected 63 ( 0.6%) items with no interactions.
EvaluatorHoldout: Processed 69643 (100.0%) in 2.95 min. Users per second: 393


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.010149,0.0111,0.004839,0.00283,0.003056,0.024269,0.005407,0.006554,0.089442,0.026144,...,0.032113,0.996637,0.089141,0.011345,7.146886,0.987461,0.058263,0.631064,0.226407,0.136275


## A hybrid content-collaborative model

In order to account for both the collaborative and content-based data, a new ICM must be created in which the itemID is also present. The ItemID becomes a feature, this can be done by concatenating the ICM with a simple diagonal matrix of shape |n_items x n_items|. The same happens for UserIDs in the UCM.

#### Challenges
* FM are not particularly easy to optimize
* If the content-based data has poor quality, the FM will simply only use the collaborative information. Providing a high weight for the content data may help in this regard
* Using a large number of embeddings rapidly increases the training time and the number of parameters

In [9]:
import scipy.sparse as sps

class LightFMItemHybridRecommender(LightFMCBFRecommender):
    """LightFMItemHybridRecommender"""

    RECOMMENDER_NAME = "LightFMItemHybridRecommender"

    def __init__(self, URM_train, ICM_train):
        super(LightFMItemHybridRecommender, self).__init__(URM_train, ICM_train)

        # Need to hstack item_features to ensure each ItemIDs are present in the model
        eye = sps.eye(self.n_items, self.n_items).tocsr()
        self.ICM_train = sps.hstack((eye, self.ICM_train)).tocsr()

In [10]:
recommender = LightFMItemHybridRecommender(URM_train, ICM_genres)
recommender.fit(epochs = 10)

result_df, _ = evaluator_validation.evaluateRecommender(recommender)
result_df

LightFMItemHybridRecommender: URM Detected 63 ( 0.6%) items with no interactions.
EvaluatorHoldout: Processed 69643 (100.0%) in 3.29 min. Users per second: 353


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.180479,0.218305,0.134624,0.093949,0.110002,0.382283,0.176081,0.154215,0.728673,0.588949,...,0.090628,0.996637,0.726223,0.014357,7.584035,0.991341,0.073737,0.669664,1.941037,0.088522


## Combining heterogeneous features

FactorizationMachines can also account for more varied feature types, for example:
* Other items in the user profile
* Features related to how active a user is or popular an item is
* One-hot (or multiple-hot) encoding of user groups
* (normalized) Recommendation quality of other models for that user group (e.g., knowing how good is a TopPopular for them may help the model decide to increase the importance of models with higher popularity bias, or not)
* User-item prediction computed by *other models*
this way leaving to the FM to learn the weights and interactions.

#### Challenges
It may be difficult to train a FM if you have a set of heterogeneous features. Ensure they are normalized.